In [2]:
from itertools import combinations, permutations
import time
import numpy as np
import random
import math
import itertools
import os
import sys
from skhep.math.vectors import LorentzVector

from lhereader import LHEReader


def smear(particle, PID):
    if(PID=="quark"):
        E, px, py, pz = particle.e, particle.px, particle.py, particle.pz
        sigma = 0.1
        new_E = np.random.normal(E, E*sigma)
        ratio = new_E/E
        new_px, new_py, new_pz = px*ratio, py*ratio, pz*ratio
        new_particle = LorentzVector()
        new_particle.setpxpypze(new_px, new_py, new_pz, new_E)

    return new_particle

In [3]:
#inputs
e_total = 1000

In [6]:
data_file_tth = "/Users/morgancassidy/Desktop/CPV/tth/cpv_8 3tev.lhe"  #0Newdata/tth_sm/tth_CPVSM_1TeV.lhe #/tth/cpv_3 1tev.lhe" 
data_file_tthvv = "/Users/morgancassidy/Desktop/CPV/tthvv/new lhes/tthvv 3 tev/tthvv_cpv_8.1.lhe" 
data_file_tbhmv = "/Users/morgancassidy/Desktop/CPV/tbhmv/jenny cuts/1 tev/cpv_7.1 1tev.lhe" #0Newdata/tbhmv_sm/tbhmvSM_1TeV.lhe" #"tbhmv/cpv_3 1tev.lhe"

tth_txt_parton = 'tth_cpv8_parton_3tev'
tth_txt_smear = 'tth_cpv8_smear_3tev'

tthvv_txt_parton = 'tthvv_cpv_parton_3tev'
tthvv_txt_smear = 'tthvv_cpv8.1_smear_3tev'

tbhmv_txt_parton = 'tbhmv_cpv7.1_parton_1tev(jenny cut)'
tbhmv_txt_smear = 'tbhmv_cpv7.1_smear_1tev(jenny cut)'

In [15]:
data_file_gbbgtt = "/Users/morgancassidy/Desktop/CPV/0Newdata/Bkg_gbbgtt/gbbgtt_1TeV.lhe"
data_file_ttza = "/Users/morgancassidy/Desktop/CPV/0Newdata/Bkg_ttza/mm2ttZa_Decay_1TeV.lhe"
data_file_ttbbvv = "/Users/morgancassidy/Desktop/CPV/0Newdata/bkg_ttbbvv/mm2ttbbvv_10TeV.lhe"
data_file_ww = "/Users/morgancassidy/Desktop/CPV/0Newdata/Bkg_mm2WW/mm2WW_1Tev_bkg.lhe"

gbb_txt_parton = 'gbb_sm_parton_1tev(run2)'
gbb_txt_smear = 'gbb_sm_smear_1tev(run2)'

ttza_txt_parton = 'ttza_sm_parton_1tev(run2)'
ttza_txt_smear = 'ttza_sm_smear_1tev(run2)'

ttbbvv_txt_parton = 'ttbbvv_sm_parton_10tev(run2)'
ttbbvv_txt_smear = 'ttbbvv_sm_smear_10tev(run2)'

ww_txt_parton = 'ww_sm_parton_1tev(run2)'
ww_txt_smear = 'ww_sm_smear_1tev(run2)'

In [54]:
reader_tth = LHEReader(data_file_tth)

result1_tth, result2_tth = [], []

for count, event in enumerate(reader_tth):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()     
        if (particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()    
        if (particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()            
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()  
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()  
        if ((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16) and particle.parent == wm_i):
            neu1 = particle.p4()         
        if ((abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15) and particle.parent == wm_i):
            l1 = particle.p4()
    
    b1_smear = smear(b1, PID="quark") 
    b2_smear = smear(b2, PID="quark")
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark")
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark")
    h_smear = b3_smear + b4_smear
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear
    tm_smear = b2_smear + neu1 + l1        
        
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark") 
        
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 

    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark")         
 
    while not (0 < h_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
        while not (0 < b3_smear[3] < e_total/2):
            b3_smear = smear(b3_smear, PID="quark") 
        b4_smear = smear(b4_smear, PID="quark") 
        while not (0 < b4_smear[3] < e_total/2):
            b4_smear = smear(b4_smear, PID="quark")           
        h_smear = b3_smear + b4_smear

        
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
        

    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark") 
        
                
    while not (0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark") 
        w_smear = j1_smear + j2_smear
        
    
    while not (0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark") 
        j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")        
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")             
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear            
        t_smear = b1_smear + w_smear


    while not ( 0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID='quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")         
        tm_smear = b2_smear + neu1 + l1
        
#________________________________________________________    
    
    if (tm_smear[3] + t_smear[3] + h_smear[3] < e_total):        
        result1_tth.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
        result2_tth.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
   
    else:
        while (tm_smear[3] + t_smear[3] + h_smear[3] > e_total):
            b1_smear = smear(b1, PID = 'quark')
            b2_smear = smear(b2, PID = 'quark')
            b3_smear = smear(b3, PID = 'quark')
            b4_smear = smear(b4, PID = 'quark')
            j1_smear = smear(j1, PID = 'quark')
            j2_smear = smear(j2, PID = 'quark')
            h_smear = b3_smear + b4_smear
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear
            tm_smear = b2_smear + neu1 + l1
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark") 
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark")
                
            while not (0 < h_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark")
                while not (0 < b3_smear[3] < e_total/2):
                    b3_smear = smear(b3_smear, PID="quark") 
                b4_smear = smear(b4_smear, PID="quark") 
                while not (0 < b4_smear[3] < e_total/2):
                    b4_smear = smear(b4_smear, PID="quark")           
                h_smear = b3_smear + b4_smear


            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 


            while not (0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark") 
                w_smear = j1_smear + j2_smear


            while not (0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark") 
                j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")        
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")             
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark") 
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear
                t_smear = b1_smear + w_smear


            while not ( 0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID='quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")         
                tm_smear = b2_smear + neu1 + l1
                    
        result1_tth.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_tth.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
    
       

result1_tth, result2_tth = np.array(result1_tth), np.array(result2_tth)

np.savetxt(tth_txt_parton, result1_tth)
np.savetxt(tth_txt_smear, result2_tth)

In [35]:
reader_tthvv = LHEReader(data_file_tthvv)

result1_tthvv, result2_tthvv = [], []

for count, event in enumerate(reader_tthvv):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()     
        if (particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()    
        if (particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()            
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()  
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()  
        if ((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16) and particle.parent == wm_i):
            neu1 = particle.p4()         
        if ((abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15) and particle.parent == wm_i):
            l1 = particle.p4()
    
    b1_smear = smear(b1, PID="quark") 
    b2_smear = smear(b2, PID="quark")
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark")
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark")
    h_smear = b3_smear + b4_smear
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear
    tm_smear = b2_smear + neu1 + l1        
    
    #smear all b jets
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark") 
        
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 

    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark")         
     
    #smear reconstructed higgs
    while not (0 < h_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
        while not (0 < b3_smear[3] < e_total/2):
            b3_smear = smear(b3_smear, PID="quark") 
        b4_smear = smear(b4_smear, PID="quark") 
        while not (0 < b4_smear[3] < e_total/2):
            b4_smear = smear(b4_smear, PID="quark")           
        h_smear = b3_smear + b4_smear

    #smear all jets    
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
        

    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark") 
        
    #smear reconstructed w            
    while not (0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark") 
        w_smear = j1_smear + j2_smear
        
    #smear reconstructed top (from w --> j1 j2 and b1)
    while not (0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark") 
        j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")        
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")             
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear            
        t_smear = b1_smear + w_smear


    while not ( 0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID='quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")         
        tm_smear = b2_smear + neu1 + l1
        
#________________________________________________________    
    
    if (tm_smear[3] + t_smear[3] + h_smear[3] < e_total):        
        result1_tthvv.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz, 
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
        result2_tthvv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz, 
                   neu1.e, neu1.px, neu1.py, neu1.pz,])
   
    else:
        while (tm_smear[3] + t_smear[3] + h_smear[3] > e_total):
            b1_smear = smear(b1, PID = 'quark')
            b2_smear = smear(b2, PID = 'quark')
            b3_smear = smear(b3, PID = 'quark')
            b4_smear = smear(b4, PID = 'quark')
            j1_smear = smear(j1, PID = 'quark')
            j2_smear = smear(j2, PID = 'quark')
            h_smear = b3_smear + b4_smear
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear
            tm_smear = b2_smear + neu1 + l1
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark") 
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark")  
                
                
            while not (0 < h_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
                while not (0 < b3_smear[3] < e_total/2):
                    b3_smear = smear(b3_smear, PID="quark") 
                b4_smear = smear(b4_smear, PID="quark") 
                while not (0 < b4_smear[3] < e_total/2):
                    b4_smear = smear(b4_smear, PID="quark")           
                h_smear = b3_smear + b4_smear


            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 


            while not (0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark") 
                w_smear = j1_smear + j2_smear


            while not (0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark") 
                j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")        
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")             
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark") 
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear           
                t_smear = b1_smear + w_smear


            while not ( 0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID='quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")         
                tm_smear = b2_smear + neu1 + l1
                    
        result1_tthvv.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_tthvv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz, 
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz, 
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
    
       

#result1_tthvv, result2_tthvv = np.array(result1_tthvv), np.array(result2_tthvv)


np.savetxt(tthvv_txt_parton, result1_tthvv)
np.savetxt(tthvv_txt_smear, result2_tthvv)

In [7]:
reader_tbhmv = LHEReader(data_file_tbhmv)

result1_tbhmv, result2_tbhmv = [], []

for count, event in enumerate(reader_tbhmv):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== 25):
            higgs_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid == 13):
            mu_i = ipar
        elif (particle.pdgid == -13):
            mum_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()    
        if (particle.pdgid== -5 and particle.parent != higgs_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent == higgs_i ):
            b3 = particle.p4()
        if (particle.pdgid== -5 and particle.parent == higgs_i):
            b4 = particle.p4()
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()
        if ((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16)):# and particle.parent == mu_i and particle.parent == mum_i):
            neu1 = particle.p4()      
        if ((abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15)): #and particle.parent == mu_i and particle.parent == mum_i):
            l1 = particle.p4() 
     
    
    b1_smear = smear(b1, PID = 'quark')
    b2_smear = smear(b2, PID = 'quark')
    b3_smear = smear(b3, PID = 'quark')
    b4_smear = smear(b4, PID = 'quark')
    j1_smear = smear(j1, PID = 'quark')
    j2_smear = smear(j2, PID = 'quark')        
    h_smear = b3_smear + b4_smear
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear
    
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark")     
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark")
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark") 
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark")        
   
    while not (0 < h_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
        while not (0 < b3_smear[3] < e_total/2):
            b3_smear = smear(b3_smear, PID="quark")         
        b4_smear = smear(b4_smear, PID="quark")
        while not (0 < b4_smear[3] < e_total/2):
            b4_smear = smear(b4_smear, PID="quark")   
            
        h_smear = b3_smear + b4_smear
            
    while not (0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")         
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark") 
            
        w_smear = j1_smear + j2_smear
  
    
    while not (0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark")         
        j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
            
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")         
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear
            
        t_smear = b1_smear + w_smear

        
#---------------------------------------------------------------

    if (b2_smear[3] + l1[3] + t_smear[3] + h_smear[3] < e_total):
        result1_tbhmv.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz, 
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz, 
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_tbhmv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

    else:
        while (b2_smear[3] + l1[3] + t_smear[3] + h_smear[3] > e_total):
            b1_smear = smear(b1, PID = 'quark')
            b2_smear = smear(b2, PID = 'quark')
            b3_smear = smear(b3, PID = 'quark')
            b4_smear = smear(b4, PID = 'quark')
            j1_smear = smear(j1, PID = 'quark')
            j2_smear = smear(j2, PID = 'quark')        
            h_smear = b3_smear + b4_smear
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear

            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark")     
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark")
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark")        

            while not (0 < h_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
                while not (0 < b3_smear[3] < e_total/2):
                    b3_smear = smear(b3_smear, PID="quark")         
                b4_smear = smear(b4_smear, PID="quark")
                while not (0 < b4_smear[3] < e_total/2):
                    b4_smear = smear(b4_smear, PID="quark")        
                h_smear = b3_smear + b4_smear

            while not (0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")         
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark") 
                w_smear = j1_smear + j2_smear


            while not (0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark")         
                j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                    
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark")         
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear  
                    
                t_smear = b1_smear + w_smear

        result1_tbhmv.append([b1.e, b1.px, b1.py, b1.pz,
                           b2.e, b2.px, b2.py, b2.pz, 
                           b3.e, b3.px, b3.py, b3.pz,
                           b4.e, b4.px, b4.py, b4.pz,
                           j1.e, j1.px, j1.py, j1.pz,
                           j2.e, j2.px, j2.py, j2.pz,
                           l1.e, l1.px, l1.py, l1.pz,
                           neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_tbhmv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                           b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                           b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                           b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                           j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                           j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz, 
                           l1.e, l1.px, l1.py, l1.pz,
                           neu1.e, neu1.px, neu1.py, neu1.pz])



result1_tbhmv, result2_tbhmv = np.array(result1_tbhmv), np.array(result2_tbhmv)


np.savetxt(tbhmv_txt_parton, result1_tbhmv)
np.savetxt(tbhmv_txt_smear, result2_tbhmv)

In [13]:
reader_gbbgtt = LHEReader(data_file_gbbgtt)

result1_gbb, result2_gbb = [], []

for count, event in enumerate(reader_gbbgtt):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if(particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()          
        if(particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()                     
        if(particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()
        if(particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()
        if(particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4() 
        if(particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()                
        if((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16) and particle.parent == wm_i):
            neu1 = particle.p4()                
        if(abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15 ):
            l1 = particle.p4()
  
            
    b1_smear = smear(b1, PID="quark")  
    b2_smear = smear(b2, PID="quark") 
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark") 
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark")
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear  
    tm_smear = b2_smear + neu1 + l1
    
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark")     
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark")
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark") 
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark")     

    while not (0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear   

    while not (0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark")
        j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")         
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear  
                    
        t_smear = b1_smear + w_smear 

        
    while not (0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID = 'quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")
        tm_smear = b2_smear + neu1 + l1


    if (tm_smear[3] + t_smear[3] + b3_smear[3] + b4_smear[3] < e_total):
        result1_gbb.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_gbb.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz, 
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz, 
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
                
    else:
        while(tm_smear[3] + t_smear[3] + b3_smear[3] + b4_smear[3] > e_total):
            b1_smear = smear(b1, PID="quark")  
            b2_smear = smear(b2, PID="quark") 
            b3_smear = smear(b3, PID="quark")
            b4_smear = smear(b4, PID="quark") 
            j1_smear = smear(j1, PID="quark")
            j2_smear = smear(j2, PID="quark")
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear  
            tm_smear = b2_smear + neu1 + l1
            
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark")     
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark")
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark")     

            while not (0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear   

            while not (0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark")
                j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark")         
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear  
                    
                t_smear = b1_smear + w_smear
                
            while not (0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID = 'quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")
                tm_smear = b2_smear + neu1 + l1

                
        result1_gbb.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz, 
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz, 
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_gbb.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz, 
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])


result1_gbb, result2_gbb = np.array(result1_gbb), np.array(result2_gbb)


np.savetxt(gbb_txt_parton, result1_gbb)
np.savetxt(gbb_txt_smear, result2_gbb)

In [21]:
reader_ttza = LHEReader(data_file_ttza)

result1_ttza, result2_ttza = [], []


for count, event in enumerate(reader_ttza):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()      
        if (particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()
        if (particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()  
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()    
        if (abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16 and particle.parent == wm_i):
            neu1 = particle.p4()                 
        if (abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15 ):
            l1 = particle.p4()
            
    b1_smear = smear(b1, PID="quark")
    b2_smear = smear(b2, PID="quark")
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark")
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark") 
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear
    tm_smear = b2_smear + l1 + neu1
        
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark")     
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark")
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark") 
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark")  
            
    while not(0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear , PID='quark')
        j2_smear = smear(j2_smear , PID='quark')
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear
        
    while not(0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark")
        j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")         
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear  
                    
        t_smear = b1_smear + w_smear

            
    while not(0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID='quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")
        tm_smear = b2_smear + l1 + neu1

            
    if (b3_smear[3]+b4_smear[3] + t_smear[3] + tm_smear[3] < e_total):
        result1_ttza.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz, 
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_ttza.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz, 
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz, 
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
        
    else:
        while (b3_smear[3]+ b4_smear[3] + t_smear[3] + tm_smear[3] > e_total):
            b1_smear = smear(b1, PID="quark")
            b2_smear = smear(b2, PID="quark")
            b3_smear = smear(b3, PID="quark")
            b4_smear = smear(b4, PID="quark")
            j1_smear = smear(j1, PID="quark")
            j2_smear = smear(j2, PID="quark") 
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear
            tm_smear = b2_smear + l1 + neu1
            
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark")     
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark")
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark")  

            while not(0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear , PID='quark')
                j2_smear = smear(j2_smear , PID='quark')
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear

            while not(0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark")
                j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear
                
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark")         
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear  
                    
                t_smear = b1_smear + w_smear


            while not(0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID='quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")
                tm_smear = b2_smear + l1 + neu1
                
        result1_ttza.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz, 
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_ttza.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz, 
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz, 
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])


result1_ttza, result2_ttza = np.array(result1_ttza), np.array(result2_ttza)


np.savetxt(ttza_txt_parton, result1_ttza)
np.savetxt(ttza_txt_smear, result2_ttza)

In [16]:
reader_ttbbvv = LHEReader(data_file_ttbbvv)

result1_ttbbvv, result2_ttbbvv = [], []

for count, event in enumerate(reader_ttbbvv):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()     
        if (particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()    
        if (particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()            
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()  
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()  
        if ((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16) and particle.parent == wm_i):
            neu1 = particle.p4()         
        if ((abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15) and particle.parent == wm_i):
            l1 = particle.p4()
    
    b1_smear = smear(b1, PID="quark") 
    b2_smear = smear(b2, PID="quark")
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark")
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark")
    h_smear = b3_smear + b4_smear
    w_smear = j1_smear + j2_smear
    t_smear = w_smear + b1_smear
    tm_smear = b2_smear + neu1 + l1        
    
    #smear all b jets
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark") 
        
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 

    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark")         
     
    #smear reconstructed higgs
    while not (0 < h_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
        while not (0 < b3_smear[3] < e_total/2):
            b3_smear = smear(b3_smear, PID="quark") 
        b4_smear = smear(b4_smear, PID="quark") 
        while not (0 < b4_smear[3] < e_total/2):
            b4_smear = smear(b4_smear, PID="quark")           
        h_smear = b3_smear + b4_smear

    #smear all jets    
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
        

    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark") 
        
    #smear reconstructed w            
    while not (0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark") 
        w_smear = j1_smear + j2_smear
        
    #smear reconstructed top (from w --> j1 j2 and b1)
    while not (0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark") 
        j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")        
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")             
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark")
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear            
        t_smear = b1_smear + w_smear


    while not ( 0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID='quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")         
        tm_smear = b2_smear + neu1 + l1
        
#________________________________________________________    
    
    if (tm_smear[3] + t_smear[3] + h_smear[3] < e_total):        
        result1_ttbbvv.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz, 
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
        result2_ttbbvv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz, 
                   neu1.e, neu1.px, neu1.py, neu1.pz,])
   
    else:
        while (tm_smear[3] + t_smear[3] + h_smear[3] > e_total):
            b1_smear = smear(b1, PID = 'quark')
            b2_smear = smear(b2, PID = 'quark')
            b3_smear = smear(b3, PID = 'quark')
            b4_smear = smear(b4, PID = 'quark')
            j1_smear = smear(j1, PID = 'quark')
            j2_smear = smear(j2, PID = 'quark')
            h_smear = b3_smear + b4_smear
            w_smear = j1_smear + j2_smear
            t_smear = w_smear + b1_smear
            tm_smear = b2_smear + neu1 + l1
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark") 
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark")  
                
                
            while not (0 < h_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
                while not (0 < b3_smear[3] < e_total/2):
                    b3_smear = smear(b3_smear, PID="quark") 
                b4_smear = smear(b4_smear, PID="quark") 
                while not (0 < b4_smear[3] < e_total/2):
                    b4_smear = smear(b4_smear, PID="quark")           
                h_smear = b3_smear + b4_smear


            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 


            while not (0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark") 
                w_smear = j1_smear + j2_smear


            while not (0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark") 
                j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")        
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")             
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark")
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark") 
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear           
                t_smear = b1_smear + w_smear


            while not ( 0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID='quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")         
                tm_smear = b2_smear + neu1 + l1
                    
        result1_ttbbvv.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_ttbbvv.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz, 
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz, 
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz, 
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])
    
       

#result1_tthvv, result2_tthvv = np.array(result1_tthvv), np.array(result2_tthvv)


np.savetxt(ttbbvv_txt_parton, result1_ttbbvv)
np.savetxt(ttbbvv_txt_smear, result2_ttbbvv)

In [6]:
reader_ww = LHEReader(data_file_ww)

result1_ww, result2_ww = [], []


for count, event in enumerate(reader_ww):
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 6):
            t_i = ipar
        elif (particle.pdgid== -6):
            tbar_i = ipar
        elif (particle.pdgid== 24):
            wp_i = ipar
        elif (particle.pdgid== -24):
            wm_i = ipar
    for ipar, particle in enumerate(event.particles):
        if (particle.pdgid== 5 and particle.parent == t_i):
            b1 = particle.p4()
        if (particle.pdgid== -5 and particle.parent == tbar_i):
            b2 = particle.p4()
        if (particle.pdgid== 5 and particle.parent != t_i and particle.parent != tbar_i):
            b3 = particle.p4()
        if (particle.pdgid== -5 and particle.parent != t_i and particle.parent != tbar_i):
            b4 = particle.p4()
        if (particle.parent == wp_i and particle.pdgid>0):
            j1 = particle.p4()       
        if (particle.parent == wp_i and particle.pdgid<0):
            j2 = particle.p4()
        if ((abs(particle.pdgid)== 12 or abs(particle.pdgid)== 14 or abs(particle.pdgid)== 16) and particle.parent == wm_i):
            neu1 = particle.p4()        
        if (abs(particle.pdgid)== 11 or abs(particle.pdgid)== 13 or abs(particle.pdgid)== 15 ):
            l1 = particle.p4()
    

    b1_smear = smear(b1, PID="quark")        
    b2_smear = smear(b2, PID="quark")  
    b3_smear = smear(b3, PID="quark")
    b4_smear = smear(b4, PID="quark")
    j1_smear = smear(j1, PID="quark")
    j2_smear = smear(j2, PID="quark")
    w_smear = j1_smear + j2_smear
    t_smear = b1_smear + w_smear
    tm_smear = l1 + neu1 + b2_smear
    
    while not (0 < b1_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark")     
    while not (0 < b2_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID="quark")
    while not (0 < b3_smear[3] < e_total/2):
        b3_smear = smear(b3_smear, PID="quark") 
    while not (0 < b4_smear[3] < e_total/2):
        b4_smear = smear(b4_smear, PID="quark") 
    while not (0 < j1_smear[3] < e_total/2):
        j1_smear = smear(j1_smear, PID="quark") 
    while not (0 < j2_smear[3] < e_total/2):
        j2_smear = smear(j2_smear, PID="quark")  
            
    while not(0 < w_smear[3] < e_total/2):
        j1_smear = smear(j1_smear , PID='quark')
        j2_smear = smear(j2_smear , PID='quark')
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear
        
    while not(0 < t_smear[3] < e_total/2):
        b1_smear = smear(b1_smear, PID="quark") 
        while not (0 < b1_smear[3] < e_total/2):
            b1_smear = smear(b1_smear, PID="quark")
        j1_smear = smear(j1_smear, PID="quark") 
        j2_smear = smear(j2_smear, PID="quark")
        while not (0 < j1_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
        while not (0 < j2_smear[3] < e_total/2):
            j2_smear = smear(j2_smear, PID="quark")
        w_smear = j1_smear + j2_smear
        while not (0 < w_smear[3] < e_total/2):
            j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark")         
            j2_smear = smear(j2_smear, PID="quark")
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark") 
            w_smear = j1_smear + j2_smear        
        t_smear = b1_smear + w_smear
            
    while not(0 < tm_smear[3] < e_total/2):
        b2_smear = smear(b2_smear, PID='quark')
        while not (0 < b2_smear[3] < e_total/2):
            b2_smear = smear(b2_smear, PID="quark")
        tm_smear = b2_smear + l1 + neu1

    if (b3_smear[3]+ b4_smear[3] + tm_smear[3] + t_smear[3] < e_total):
        result1_ww.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz, 
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_ww.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

    else:
        while(tm_smear[3] + t_smear[3] + b3_smear[3] + b4_smear[3] > e_total):
            b1_smear = smear(b1, PID='quark')
            b2_smear = smear(b2, PID='quark')
            b3_smear = smear(b3, PID='quark')
            b4_smear = smear(b4, PID='quark')
            j1_smear = smear(j1, PID='quark')
            j2_smear = smear(j2, PID='quark')
            w_smear = j1_smear + j2_smear
            t_smear = b1_smear + w_smear
            tm_smear = l1 + neu1 + b2_smear
            
            while not (0 < b1_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark")     
            while not (0 < b2_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID="quark")
            while not (0 < b3_smear[3] < e_total/2):
                b3_smear = smear(b3_smear, PID="quark") 
            while not (0 < b4_smear[3] < e_total/2):
                b4_smear = smear(b4_smear, PID="quark") 
            while not (0 < j1_smear[3] < e_total/2):
                j1_smear = smear(j1_smear, PID="quark") 
            while not (0 < j2_smear[3] < e_total/2):
                j2_smear = smear(j2_smear, PID="quark")  

            while not(0 < w_smear[3] < e_total/2):
                j1_smear = smear(j1_smear , PID='quark')
                j2_smear = smear(j2_smear , PID='quark')
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear

            while not(0 < t_smear[3] < e_total/2):
                b1_smear = smear(b1_smear, PID="quark") 
                while not (0 < b1_smear[3] < e_total/2):
                    b1_smear = smear(b1_smear, PID="quark")
                j1_smear = smear(j1_smear, PID="quark") 
                j2_smear = smear(j2_smear, PID="quark")
                while not (0 < j1_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                while not (0 < j2_smear[3] < e_total/2):
                    j2_smear = smear(j2_smear, PID="quark")
                w_smear = j1_smear + j2_smear
                while not (0 < w_smear[3] < e_total/2):
                    j1_smear = smear(j1_smear, PID="quark") 
                    while not (0 < j1_smear[3] < e_total/2):
                        j1_smear = smear(j1_smear, PID="quark")         
                    j2_smear = smear(j2_smear, PID="quark")
                    while not (0 < j2_smear[3] < e_total/2):
                        j2_smear = smear(j2_smear, PID="quark") 
                    w_smear = j1_smear + j2_smear  
                    
                t_smear = b1_smear + w_smear

            while not(0 < tm_smear[3] < e_total/2):
                b2_smear = smear(b2_smear, PID='quark')
                while not (0 < b2_smear[3] < e_total/2):
                    b2_smear = smear(b2_smear, PID="quark")
                tm_smear = b2_smear + l1 + neu1

            
        result1_ww.append([b1.e, b1.px, b1.py, b1.pz,
                   b2.e, b2.px, b2.py, b2.pz,
                   b3.e, b3.px, b3.py, b3.pz,
                   b4.e, b4.px, b4.py, b4.pz,
                   j1.e, j1.px, j1.py, j1.pz,
                   j2.e, j2.px, j2.py, j2.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])

        result2_ww.append([b1_smear.e, b1_smear.px, b1_smear.py, b1_smear.pz,
                   b2_smear.e, b2_smear.px, b2_smear.py, b2_smear.pz,
                   b3_smear.e, b3_smear.px, b3_smear.py, b3_smear.pz,
                   b4_smear.e, b4_smear.px, b4_smear.py, b4_smear.pz,
                   j1_smear.e, j1_smear.px, j1_smear.py, j1_smear.pz,
                   j2_smear.e, j2_smear.px, j2_smear.py, j2_smear.pz,
                   l1.e, l1.px, l1.py, l1.pz,
                   neu1.e, neu1.px, neu1.py, neu1.pz])



result1_ww, result2_ww = np.array(result1_ww), np.array(result2_ww)

np.savetxt(ww_txt_parton, result1_ww)
np.savetxt(ww_txt_smear, result2_ww)